# GEMM on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:

# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [68]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-jiayuangu762943' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-jiayuangu762943
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 704 bytes | 2.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-jiayuangu762943
   07be112..bfb1061  main       -> origin/main
Updating 07be112..bfb1061
Fast-forward
 src/ops.py | 64 +++++++++++++++++++++++++---------------------------------------
 1 file changed, 25 insertions(+), 39 deletions(-)
/content/gdrive/MyDrive/ece5545


In [34]:
# # This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [35]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5-conv2d_dw_gpu.ipynb	README.md  tests
2-conv1d_gpu.ipynb  4-gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2. Install TVM

In [5]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.5/428.5 MB 4.1 MB/s eta 0:00:00


## 3. Check the implementation of `make_gemm_gpu_scheduler` function in `src.ops`

The function implements General Matrix Multiply (GEMM) on GPU. You should use TVM to optimize it.

Let $A \in \mathbb{R}^{m \times k}$, $W \in \mathbb{R}^{k \times n}$, and $B \in \mathbb{R}^{m \times n}$, then
$$
B = A \times W
$$
Please see the numpy matmul function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

The `make_gemm_gpu_scheduler` takes $m$, $k$, and $n$. The first matrix is $m \times k$, the second matrix is $k \times n$, and the output matrix is $m \times n$.

The function returns both the TVM scheduler and the TVM opterator for
1. Input $a$
2. Input $w$
3. Output $b$

The scheduler should be able to used to build a function with signature $func(a, w, b)$.
Please see the following cells for usage.

In [1]:
!pip install numpy==1.24.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which 

## Vanilla Code

In [11]:
import tvm
from tvm import te
def vanilla_make_gemm_gpu_scheduler(M, K, N):
     A = te.placeholder((M, K), name="A")
     B = te.placeholder((K, N), name="B")

     # TVM Matrix Multiplication using TE
     k = te.reduce_axis((0, K), "k")
     A = te.placeholder((M, K), name="A")
     B = te.placeholder((K, N), name="B")
     C = te.compute((M, N), lambda x, y: te.sum(A[x, k] * B[k, y], axis=k), name="C")
     # Default schedule
     s = te.create_schedule(C.op)

     # the i-th block is indexed by blockIdx.x.
     # the number of threads in each block is blockDim.x
     # and the i-th thread within a block is indexed by threadIdx.x
     # overall index of a thread can be calculated as
     # 𝑖=blockIdx.x×blockDim.x+threadIdx.x
     block_x = te.thread_axis("blockIdx.y")
     block_y = te.thread_axis("blockIdx.x")

     x, y = s[C].op.axis
     (k,) = s[C].op.reduce_axis
     s[C].bind(y, block_y)
     s[C].bind(x, block_x)

     return s, A, B, C


## Vanilla runtime

In [82]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = vanilla_make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[512.28424 511.44678 507.24255 ... 499.75528 507.71405 519.5174 ]
 [513.6171  501.62268 502.89038 ... 497.90283 498.33096 506.67056]
 [509.03876 503.462   507.72232 ... 502.51074 506.05255 517.33453]
 ...
 [501.40363 495.00513 499.86172 ... 494.20172 490.54358 505.06915]
 [523.2278  514.9554  512.5181  ... 520.0199  519.0525  523.253  ]
 [529.64685 522.3683  526.1968  ... 517.64026 526.00134 530.5718 ]]
Output: [[512.284   511.44635 507.24258 ... 499.75513 507.71426 519.51685]
 [513.6173  501.623   502.89032 ... 497.90326 498.33084 506.67038]
 [509.03873 503.46185 507.72217 ... 502.5105  506.05194 517.3344 ]
 ...
 [501.40335 495.0052  499.8619  ... 494.20148 490.54312 505.0691 ]
 [523.22815 514.95575 512.5178  ... 520.01996 519.0525  523.25323]
 [529.64636 522.3688  526.1965  ... 517.6405  526.0016  530.5715 ]]
GEMM TVM: 87.990272 ms


## Optimized runtime

In [83]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[494.05237 497.4322  485.24713 ... 504.18054 489.75583 506.187  ]
 [488.38092 493.2765  482.4443  ... 497.54932 485.25323 500.66626]
 [496.98538 507.87112 492.7254  ... 520.0741  498.0563  512.6229 ]
 ...
 [498.74222 516.837   508.20953 ... 519.2613  501.58072 524.17676]
 [494.94547 513.0723  495.0604  ... 517.2201  498.5652  516.5736 ]
 [504.20526 514.8947  498.87036 ... 514.85266 504.3047  521.3501 ]]
Output: [[494.05203 497.43213 485.24744 ... 504.18088 489.75568 506.18747]
 [488.3813  493.27716 482.44363 ... 497.5492  485.25354 500.66586]
 [496.98566 507.87027 492.72528 ... 520.0746  498.05585 512.62286]
 ...
 [498.74185 516.837   508.2094  ... 519.2614  501.5805  524.17633]
 [494.9458  513.0728  495.06067 ... 517.2206  498.56494 516.57404]
 [504.20535 514.8949  498.8707  ... 514.85284 504.30518 521.3501 ]]
GEMM TVM: 72.050689 ms


In [84]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 32)
        blockIdx_y = T.launch_thread("blockIdx.y", 16)
        threadIdx_x = T.env_thread("threadIdx.x")
        threadIdx_y = T.env_thread("threadIdx.y")
        C_1 = T.Buffer((524288,), data=C.data)
        with T.launch_thread(threadIdx_x, 32):
            T.launch_thread(threadIdx_y, 32)
            C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y * 32 + threadIdx_y] = T.float32(0)
        for k_outer in range(512):
            T.launch_thread(threadIdx_x, 32)
            T.launch_thread(threadIdx_y, 32)
            for k_inner in range(4):
                A_1 = T.Buffer((2097152,), data=A.data)


In [85]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-jiayuangu762943
======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.5, pluggy-1.5.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-jiayuangu762943
plugins: anyio-4.9.0, typeguard-4.4.2, langsmith-0.3.23
collected 29 items                                                                                 

tests/test_gemm_gpu.py ........................F.FFF                                         [100%]

============================================= FAILURES =============================================
_____________________________________ test1_speed_torch[2000] ______________________________________

execution_number = 2000

    @pytest.mark.parametrize(
        'execution_number', [2, 10, 100, 1000, 2000, 4000, 6000, 8000, 10000])
    def test1_speed_torch(execution_number):
        # Define dimension
        M = 1024
        K = 1024
        N = 1024
        n